$\newcommand{\To}{\Rightarrow}$

In [3]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [32]:
from kernel.type import TFun, boolT
from kernel.term import Term, Var
from kernel.thm import Thm
from logic.nat import natT
from logic import basic
from syntax import printer

thy = basic.loadTheory('nat')

## Equality

In this section, we introduce rules for working with equality. Equality is characterized by four essential properties: reflexivity, symmetry, transitivity, and congruence. They are stated as follows:

$$ \frac{}{\vdash x = x} \hbox{ reflexive} $$

$$ \frac{\vdash x = y}{\vdash y = x} \hbox{ symmetric} $$

$$ \frac{\vdash x = y \quad \vdash y = z}{\vdash x = z} \hbox{ transitive} $$

$$ \frac{\vdash f = g \quad \vdash x = y}{\vdash f\ x = g\ y} \hbox{ combination} $$

These rules can be invoked as follows:

In [25]:
x = Var("x", natT)
y = Var("y", natT)
z = Var("z", natT)

print(printer.print_thm(thy, Thm.reflexive(x), unicode=True))

th1 = Thm([], Term.mk_equals(x, y))
th2 = Thm([], Term.mk_equals(y, z))
print(printer.print_thm(thy, Thm.symmetric(th1), unicode=True))

print(printer.print_thm(thy, Thm.transitive(th1, th2), unicode=True))

f = Var("f", TFun(natT, natT))
g = Var("g", TFun(natT, natT))

th5 = Thm([], Term.mk_equals(f, g))
print(printer.print_thm(thy, Thm.combination(th5, th1), unicode=True))

⊢ x = x
⊢ y = x
⊢ x = z
⊢ f x = g y


These rules can be used with `implies_intr` to prove corresponding implication theorems. The following example show a variant of the combination rule, where the function stays the same.

#### Example:

Prove $x = y \to f\ x = f\ y$.

#### Solution:

0. $\vdash f = f$ by reflexive $f$.
1. $x = y \vdash x = y$ by assume $x = y$.
2. $x = y \vdash f\ x = f\ y$ by combination from 0, 1.
3. $\vdash x = y \to f\ x = f\ y$ by implies_intr $x = y$.

In Python, this is as follows:

In [27]:
th0 = Thm.reflexive(f)
th1 = Thm.assume(Term.mk_equals(x, y))
th2 = Thm.combination(th0, th1)
th3 = Thm.implies_intr(Term.mk_equals(x, y), th2)
print(printer.print_thm(thy, th3, unicode=True))

⊢ x = y ⟶ f x = f y


We now show a more complex example, on the congruence property of functions on two variables:

#### Example:

Prove $x_1 = x_2 \to y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$.

#### Solution:

0. $\vdash h = h$ by reflexive $h$.
1. $x_1 = x_2 \vdash x_1 = x_2$ by assume $x_1 = x_2$.
2. $y_1 = y_2 \vdash y_1 = y_2$ by assume $y_1 = y_2$.
3. $x_1 = x_2 \vdash h\ x_1 = h\ x_2$ by combination from 0, 1.
4. $x_1 = x_2, y_1 = y_2 \vdash h\ x_1\ y_1 = h\ x_2\ y_2$ by combination from 3, 2.
5. $x_1 = x_2 \vdash y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$ by implies_intr $y_1 = y_2$ from 4.
6. $\vdash x_1 = x_2 \to y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$ by implies_intr $x_1 = x_2$ from 5.

In [30]:
h = Var("h", TFun(natT, natT, natT))
x1 = Var("x1", natT)
x2 = Var("x2", natT)
y1 = Var("y1", natT)
y2 = Var("y2", natT)
th0 = Thm.reflexive(h)
th1 = Thm.assume(Term.mk_equals(x1, x2))
th2 = Thm.assume(Term.mk_equals(y1, y2))
th3 = Thm.combination(th0, th1)
th4 = Thm.combination(th3, th2)
th5 = Thm.implies_intr(Term.mk_equals(y1, y2), th4)
th6 = Thm.implies_intr(Term.mk_equals(x1, x2), th5)
print(printer.print_thm(thy, th6, unicode=True))

⊢ x1 = x2 ⟶ y1 = y2 ⟶ h x1 y1 = h x2 y2


## Equality on booleans

Two primitive deduction rules concern equality between boolean terms. `equal_intr` states that an equality between boolean terms can be proved by showing two implications. This corresponds to showing an if-and-only-if statement by showing the two directions.

$$ \frac{\vdash A \to B \quad \vdash B \to A}{\vdash A = B} \hbox{ equal_intr} $$

An example:

In [34]:
A = Var("A", boolT)
B = Var("B", boolT)
th0 = Thm([], Term.mk_implies(A, B))
th1 = Thm([], Term.mk_implies(B, A))
print(printer.print_thm(thy, Thm.equal_intr(th0, th1), unicode=True))

⊢ A = B


The `equal_elim` rule states that given $A = B$ and $A$, we can obtain $B$. It shows how to make use of an equality between booleans.

$$ \frac{\vdash A = B \quad \vdash A}{\vdash B} \hbox{ equal_elim} $$

#### Example:

Prove $\vdash a = b \to P\ a \to P\ b$.

#### Solution:

0. $a = b \vdash a = b$ by assume $a = b$.
1. $P\ a \vdash P\ a$ by assume $P\ a$.
2. $\vdash P = P$ by reflexive $P$.
3. $a = b \vdash P\ a = P\ b$ by combination from 2, 0.
4. $a = b, P\ a \vdash P\ b$ by equal_elim from 3, 1.
5. $a = b \vdash P\ a \to P\ b$ by implies_intr $P\ a$.
6. $\vdash a = b \to P\ a \to P\ b$ by implies_intr $a = b$.

In [40]:
a = Var("a", natT)
b = Var("b", natT)
P = Var("P", TFun(natT, boolT))
th0 = Thm.assume(Term.mk_equals(a, b))
th1 = Thm.assume(P(a))
th2 = Thm.reflexive(P)
th3 = Thm.combination(th2, th0)
th4 = Thm.equal_elim(th3, th1)
th5 = Thm.implies_intr(P(a), th4)
th6 = Thm.implies_intr(Term.mk_equals(a, b), th5)
print(printer.print_thm(thy, th6, unicode=True))

⊢ a = b ⟶ P a ⟶ P b
